# Análisis de datos

Para generar las slides use:

In [30]:
%%bash
jupyter nbconvert ALL_inspire.ipynb --to slides --reveal-prefix "https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.1.0"

[NbConvertApp] Converting notebook ALL_inspire.ipynb to slides
[NbConvertApp] Writing 273514 bytes to ALL_inspire.slides.html


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',200)

## Load data (WARNING: 1.5 GB)
Download from [here](https://drive.google.com/file/d/1NtocdsxbTky5uLmKp3eDoGcQG7qsTEYv/view?usp=sharing) 

In [3]:
df_all=pd.read_json('hep_records.json',lines=True)

In [4]:
df_all.shape

(1271720, 10)

## Keep only entries with references

In [5]:
dfr=df_all[df_all.references.map(len)>0].reset_index(drop=True)

In [6]:
dfr.shape

(896637, 10)

### make a query

In [96]:
q=input('Search for author, e.g: restrepo, d:  ')

rq=dfr[dfr.full_authors.str.lower().str.contains('restrepo, d')].reset_index(drop=True)
print('{} records found'.format(rq.shape[0])  )
rq

Search for author, e.g: restrepo, d:  restrepo,d
46 records found


,abstract,authors,citations,co-authors,creation_date,free_keywords,recid,references,standardized_keywords,title,full_authors
0,"In the simplest scheme for neutrino masses invoking a triplet of Higgs scalars there are two CP-even neutral Higgs bosons $H_i$ (i=1,2) and one massive pseudoscalar $A$. For some choices of parame...","[Diaz, M.A.]","[468209, 614714, 1387338, 594326, 1465858]","[Garcia-Jareno, M.A., Restrepo, D.A., Valle, J.W.F.]",1997-12,[],452755,"[452289, 156195, 297673, 154794, 338828, 153987, 448471, 154264, 165562, 426586, 415276]","[electron positron: annihilation, Z0: electroproduction, Z0: width, electroweak interaction: validity test, Higgs particle: triplet, Higgs particle: mass, neutrino: mass, Higgs particle: hadronic ...",Neutrino mass and missing momentum Higgs boson signals,"Diaz, M.A.\nGarcia-Jareno, M.A.\nRestrepo, D.A.\nValle, J.W.F.\n"
1,"We perform a careful study of the neutral scalar sector of a model which includes a singlet, a doublet, and a triplet scalar field under $SU(2)$. This model is motivated by neutrino physics, since...","[Diaz, Marco A.]","[871431, 1618442, 594326, 1343128, 1454236, 818590, 733727, 511268, 496198, 871848, 1405993, 817322, 565163, 685357, 538417, 614714, 1419067, 484678, 1387338, 1477837, 611151, 475730, 1484516, 149...","[Garcia-Jareno, M.A., Restrepo, Diego A., Valle, J.W.F.]",1998-03,"[14.80.Cp, 13.85.Qk, 11.30.Fs, 12.60.Fr, 14.60.Pq]",468209,"[341121, 12291, 12292, 384903, 338828, 354450, 153987, 154264, 448153, 21019, 337948, 357919, 156195, 219303, 31786, 415276, 361648, 165562, 341307, 450497, 297673, 38603, 36949, 295257, 355805, 3...","[electron positron: annihilation, Higgs particle: electroproduction, Higgs particle: associated production, Majoron, neutrino: mass generation, electroweak interaction: model, Higgs particle: mult...",Seesaw Majoron model of neutrino mass and novel signals in Higgs boson production at LEP,"Diaz, Marco A.\nGarcia-Jareno, M.A.\nRestrepo, Diego A.\nValle, J.W.F.\n"
2,We study unconventional decays of the top-quark and the top-squark in the framework of SUSY models with broken R-parity. The model under study is the MSSM with an additional bilinear term that bre...,"[de Campos, F.]","[500354, 1325035, 618376, 1228048, 497300, 525081, 579226, 705563, 1296678, 780075, 530873, 508352, 1266784, 503363, 555590, 1234255, 784720, 561745, 509817, 510559, 500832, 505185, 1229521, 56663...","[Diaz, M.A., Eboli, Oscar J.P., Magro, M.B., Navarro, L., Porod, W., Restrepo, D.A., Valle, J.W.F.]",1998-11,[],483425,"[15873, 396546, 474677, 444170, 393099, 452880, 446097, 453140, 451477, 295257, 466802, 451738, 15646, 480287, 422690, 425251, 419885, 419377, 466483, 419381, 427318, 466366, 421951, 477316, 42771...","[talk: Batavia 1998/11/19, electroweak interaction, supersymmetry, R parity: violation, top: decay modes, squark: top, squark: decay, neutrino/tau: mass, top: branching ratio, squark: branching ra...",R-parity violating decays of the top quark and the top squark at the Tevatron,"de Campos, F.\nDiaz, M.A.\nEboli, Oscar J.P.\nMagro, M.B.\nNavarro, L.\nPorod, W.\nRestrepo, D.A.\nValle, J.W.F.\n"
3,We present an outlook for possible discovery of supersymmetry with broken R-parity at Run II of the Tevatron. We first present a review of the literature and an update of the experimental bounds. ...,"[Allanach, B.]","[607232, 607233, 563718, 601608, 667151, 1410582, 1624599, 530456, 562201, 1256960, 509103, 611357, 733727, 1395748, 612390, 733617, 805935, 887860, 538165, 1091641, 568890, 610875, 555069, 777279...","[Baer, H., Banerjee, S., Berger, Edmond L., Chertok, M.B., Cheung, King-man, de Campos, F., Dedes, A., Diaz, M.A., Dreiner, Herbert K., Eboli, Oscar J.P., Gunion, John F., Harris, B.W., Hewett, J....",1999-03,[],501153,"[30227, 453140, 466455, 429080, 478234, 478236, 37978, 480287, 394274, 441899, 419885, 419377, 466483, 419381, 396346, 448572, 421951, 455239, 322640, 427604, 371286, 398425, 393306, 424543, 30316...","[talk: Batavia 199

###  Convert to WOS-like format

In [171]:
dfr['full_authors']=dfr['authors']+dfr['co-authors']

In [172]:
dfrw=dfr.copy()
dfrw.full_authors=dfrw.full_authors.str.join('\n').str.replace('.$','\n')
#convert integer list to string list: https://stackoverflow.com/a/3590175/2268280
dfrw['refs']=dfrw.references.map(lambda x: '\n'.join( map( str,x )   )).str.replace('.$','\n')

In [177]:
dfrw[['title','full_authors','refs']][:3]#.to_excel('inspire.xlsx',index=False)

,title,full_authors,refs
0,A target beam monitor using secondary electron emission,"Johnston, Lawrence II.\nMurray, Joseph J.\nGearhart, Roger A.\nHauser, Todd\nKlein, Paul R\n",4864\n
1,Neutron-Proton Scattering Below 20-MeV,"Noyes, H.Pierr\n",47202\n43619\n47204\n46245\n47400\n45692\n438702\n48175\n458288\n3218\n48052\n46838\n9144\n1476836\n46874\n47483\n46684\n47418\n4688\n
2,Coherent Production as a Means of Determining the Spin and Parity of Bosons,"Berman, S.M.\nDrell, S.D\n",147582\n


## Build the network of references

In [7]:
1+1

2

In [ ]:
import time
def check_intersect1d(x,y):
    return np.intersect1d(x,y)

def check_not_ref(x,ref):
    return x not in ref

df=dfr.sample(1000).sort_values('recid').reset_index(drop=True)

ed=pd.DataFrame()
edges=[]

s=time.time()
Nmin=4
#===log===
f=open('kk.log','w')
f.write('')
f.close()
#=======
#DEBUG===========
#i=5000
#if True:
for i in df.index:
#================    
    #remove i
    #===log===
    f=open('kk.log','a')
    f.write('{}\n'.format(i))
    f.close()
    #=======
    print(i,end='\r')
    # Drop already analysed files
    df_not_i=df.drop( df.index[ range(0,i+1)  ] ) 
    # prepare edges DataFrame
    df_not_i=df_not_i.rename_axis({'recid':'target'}, axis = 'columns')
    #WARNING: limit the number of columns to free memory
    df_not_i=df_not_i[['target','references']]
    # Make the intersection of df.loc[i,'references' with all the others one (see previous restricion)
    df_not_i['intersect']=df_not_i['references'].map(lambda x: check_intersect1d(x,df.loc[i,'references']))
    # Restrict the number of edges
    df_not_i=df_not_i[df_not_i['intersect'].map(len)>Nmin]
    # Add source column. 
    df_not_i['source']=df.loc[i,'recid']
    # drop references columns
    ed=ed.append( df_not_i[['source','target','intersect']] ).reset_index(drop=True) 
    if ed.shape[0]>20000:
        break


print(time.time()-s)

In [195]:
#DEBUG========
i=5000
if True:
#for i in df.index:
#=============    
    #remove i
    #===log===
    f=open('kk.log','a')
    f.write('{}\n'.format(i))
    f.close()
    #=======
    print(i,end='\r')
    # Drop already analysed files
    df_not_i=df.drop( df.index[ range(0,i+1)  ] ) 
    # prepare edges DataFrame
    df_not_i=df_not_i.rename_axis({'recid':'target'}, axis = 'columns')
    #WARNING: limit the number of columns to free memory
    df_not_i=df_not_i[['target','references']]
    # Make the intersection of df.loc[i,'references' with all the others one (see previous restricion)
    df_not_i['intersect']=df_not_i['references'].map(lambda x: check_intersect1d(x,df.loc[i,'references']))
    # Restrict the number of edges
    df_not_i=df_not_i[df_not_i['intersect'].map(len)>Nmin]
    # Add source column. 
    df_not_i['source']=df.loc[i,'recid']
    # drop references columns
    ed=ed.append( df_not_i[['source','target','intersect']] ).reset_index(drop=True) 
    if ed.shape[0]>20000:
        pass#break

In [201]:
df_not_i=df.drop( df.index[ range(0,i+1)  ] ) 
df_not_i.shape

(4999, 10)

In [202]:
df_not_i=df_not_i.rename_axis({'recid':'target'}, axis = 'columns')
    #WARNING: limit the number of columns to free memory
df_not_i=df_not_i[['target','references']]
    # Make the intersection of df.loc[i,'references' with all the others one (see previous restricion)

In [203]:
df_not_i['intersect']=df_not_i.references.map(lambda x: check_intersect1d(x,df.loc[i,'references']))

In [208]:
df_not_i[df_not_i.intersect.map(len)>1]

,target,references,intersect
5193,742434,"[622208, 628099, 728926, 597893, 596486, 211976, 615305, 665611, 351118, 715407, 504464, 913532, 544659, 128277, 335641, 596510, 651295, 361760, 647460, 603949, 729779, 34357, 382134, 612284, 7085...","[128277, 211976]"
7022,1088938,"[810880, 622852, 632709, 211976, 622217, 128277, 520472, 665117, 843935, 779815, 259272, 751923, 377909, 593722, 622267, 595388, 225845, 312643, 292552, 573643, 439250, 139219, 578136, 654427, 866...","[128277, 211976]"
7583,1239108,"[795755, 497668, 621061, 493703, 211976, 834185, 884237, 834964, 128277, 838428, 749214, 828321, 652580, 917991, 707625, 439339, 813613, 225845, 548664, 677051, 800574, 275527, 647752, 692937, 617...","[128277, 211976]"


In [163]:
df_not_i=df.drop( df.index[ range(0,i+1)  ] ) 

In [168]:
df.loc[i,'references']

[1433899,
 33542,
 1440262,
 68753,
 47645,
 72990,
 45472,
 9380,
 9381,
 47910,
 66345,
 66346,
 79147,
 55604,
 78136,
 79034,
 3901,
 48117,
 1435974,
 51023,
 942928,
 1433445,
 1474785,
 45541,
 79984,
 84210,
 55669,
 70905,
 65661]

In [161]:
df[:10]

,abstract,authors,citations,co-authors,creation_date,free_keywords,recid,references,standardized_keywords,title
0,The assertion that the Jost function of ans-wave nonlocal potential is equivalent to the ratio of the Fredholm determinants of the integral equations corresponding to the physical and regular solu...,"[Ahmad, S.S.]","[168224, 213603, 128614, 176042, 118224, 124049, 151868, 151934]",[],1974-01,[],687,"[1433899, 33542, 1440262, 68753, 47645, 72990, 45472, 9380, 9381, 47910, 66345, 66346, 79147, 55604, 78136, 79034, 3901, 48117, 1435974, 51023, 942928, 1433445, 1474785, 45541, 79984, 84210, 55669...","[potential scattering, partial wave, axiomatic field theory, s-matrix, analytic properties]",On the Connection Between Fredholm Determinant and Jost Function for Nonlocal Potentials
1,,"[Balazs, Louis A.P.]",[],[],1974-06,[],1238,"[93296, 84354, 84923, 81796, 56421, 72039, 84716, 81194, 73452, 68528, 93074, 93075, 85338, 67131, 84924, 84746]",[],Nondiffractive Component of Pomeranchuk Exchange at Small t in an Approximately Unitarized Dual pi pi Model
2,,"[Olesen, P.]",[],[],1974-07,[],1307,"[80965, 81351, 61163, 81230, 81238, 68024, 83804, 95133]","[LECTURES, QUARK: CONFINEMENT, MODEL: GLUON, DUALITY, MODEL: STRING, FIELD THEORETICAL MODEL, BARYON: MULTIPLET, MULTIPLET: BARYON]",Dual Confinement of Quarks
3,,"[Mignani, R.]","[122168, 99841, 123060, 98677, 100007]","[Recami, E.]",1974-08,[],1507,"[4677, 80614, 83146, 92887, 1075, 96052, 50007, 80668]",[],Possible Experimental Behavior of Tachyon Monopoles
4,,"[Zachariasen, F.]",[],[],1974-09,[],1591,"[75684, 3175, 62312, 75052, 61582, 68528, 74387, 68215, 601]",[],Selfconsistent Models of the Pomeron
5,"At the six lowest-lying levels, the quark model predicts a sequence L=0, 1, 2, … of meson states with specific JPC coupling to two and three pions. The symmetric-group model gives exactly the coin...","[Frampton, P.H.]","[99888, 1957, 263366]",[],1974-11,[],1723,"[11880, 91648, 68388, 67206, 91825]",[],Predictions of Quark and Dual Approaches to Meson Classification
6,A free motion of a relativistic string is quantized by the use ofr Dirac's generalized canonical formalism. It is shown that only the light-like gauge is applied to quantization of this system and...,"[Chikashige, Yuichi]","[18720, 203233, 120354, 14787, 8903, 9960, 7341, 3823, 108082, 120355, 3599, 15803]","[Kamimura, Kiyoshi]",1974-11,[],1738,"[80965, 42727, 26316, 9134, 3729, 2423, 68024, 54961]","[QUANTUM MECHANICS: RELATIVISTIC, MODEL: STRING, INVARIANCE: GAUGE]",The Canonical Quantization of Relativistic String
7,,"[Inagaki, Hisakazu]",[],[],1974-12,[],1864,"[1384659, 67305, 81023, 84012, 67793, 69267, 1973, 69595, 67166, 80981]",[],Extended Light Cone Algebra and electron - Proton Scattering
8,We have observed fourteen events in which two muons are produced by high-energy neutrino and antineutrino interactions. The absence of trimuon events and the observed characteristics of the dimuon...,"[Benvenuti, A.C.]","[116742, 147463, 112649, 129037, 117763, 4118, 110619, 110624, 118817, 2083, 194567, 100396, 108589, 112694, 98364, 135236, 110661, 315463, 100429, 2129, 116819, 116820, 114774, 108632, 2145, 1127...","[Cline, D., Ford, William T., Imlay, R., Ling, T.Y., Mann, A.K., Messing, F., Orr, R., Reeder, D.D., Rubbia, C., Stefanski, R., Sulak, L., Wanderer, Peter J.]",1975-02,[],1953,"[2179, 98021, 82859, 90414, 1776, 93430, 48216, 1887]",[],Observation of New Particle Production by High-Energy Neutrinos and anti-neutrinos
9,,"[Saenz, A.W.]",[],"[Zachary, W.W.]",1975-02,[],1981,"[89121, 79942, 52774, 79817, 79946, 94476, 79057, 82418, 72724, 1477126, 65720, 88217, 71546, 66239]",[],On the Equality of s Operators Corresponding to Unitarily Equivalent Hamiltonians in Single Channel Scattering


In [158]:
ed[:10]

,source,target,intersect


In [ ]:
s=time.time()
Nmin=4
#DEBUG========
#if True:
#    i=0
for i in df.index:
#=============    
    #remove i
    #===log===
    f=open('kk.log','a')
    f.write('{}\n'.format(i))
    f.close()
    #=======
    print(i,end='\r')
    df_not_i=df.drop(df.index[[i]])
    df_not_i['intersect']=df_not_i.references.map(lambda x: check_intersect1d(x,df.loc[i,'references']))
    df_not_i=df_not_i[df_not_i.intersect.map(len)>Nmin]
    df_not_i=df_not_i.rename_axis({'recid':'target'}, axis = 'columns')
    df_not_i['source']=df.loc[i,'recid']
    # switch source, target order and remove if already in edges
    df_not_i=df_not_i[ ( df_not_i.target.map( lambda x: [x] )+df_not_i.source.map( lambda x: [x] )
          ).map(lambda x: check_not_ref(x,edges) )]        
    # list of edges: [ [source, target],... ]
    try:
        edges=edges+np.stack(( df_not_i.source.values,df_not_i.target.values),axis=1 ).tolist()
        #Store Edges DataFrame
        ed=ed.append( df_not_i[['source','target','intersect']] ).reset_index(drop=True)
    except AttributeError:
        f=open('DEBUG','a')
        f.write('{}\n'.format(i))
        f.close()
        

print(time.time()-s)

In [144]:
ed_bkp=ed.copy()

In [24]:
ed.shape

(258600, 3)

In [27]:
len( edges )

258600

In [30]:
kk=ed[ed.intersect.map(len)>4]

In [34]:
kk.shape

(9277, 3)

In [37]:
kk.drop_duplicates(['source','target'])[-10:]

,source,target,intersect
258215,893812,1083623,"[399009, 645666, 666221, 750597, 792157]"
258264,893887,1344291,"[344376, 586746, 617287, 644478, 724663, 790952, 806293]"
258326,894076,1227984,"[4328, 51188, 119585, 393084, 393099]"
258328,894076,1231053,"[4328, 11883, 40208, 51188, 393084, 393099, 796887]"
258360,894076,1339957,"[4328, 11883, 40208, 51188, 1614070]"
258363,894076,1346600,"[4328, 11883, 51188, 119585, 393084, 393099]"
258381,894076,1429549,"[4328, 11883, 51188, 119585, 393084, 393099, 713909, 796887]"
258404,894076,1605397,"[455671, 713909, 796887, 1614057, 1614070]"
258412,894076,1657128,"[4328, 11883, 51188, 393084, 537586, 796887]"
258549,894529,1357569,"[412894, 441035, 520945, 832459, 865383, 865635]"


Fix edge

In [551]:
edf=ed[ed.intersect.map(len)>4].reset_index(drop=True)
edf[:3]

,source,target,intersect
0,1,2,"[2, 4, 18, 20, 25, 29, 31, 33, 34, 36, 40, 41, 51, 64, 81, 84, 86, 89, 91, 94, 95, 96, 99]"
1,1,4,"[1, 2, 4, 5, 6, 10, 15, 18, 20, 25, 30, 37, 38, 40, 44, 46, 49, 51, 53, 54, 55, 56, 71, 81, 82, 84, 86, 89, 91, 92, 96, 97]"
2,1,5,"[1, 2, 4, 6, 10, 12, 13, 15, 17, 20, 21, 22, 25, 29, 31, 37, 38, 46, 49, 52, 54, 55, 58, 64, 67, 70, 81, 82, 84, 89, 91, 92, 94, 95]"


In [552]:
edf

,source,target,intersect
0,1,2,"[2, 4, 18, 20, 25, 29, 31, 33, 34, 36, 40, 41, 51, 64, 81, 84, 86, 89, 91, 94, 95, 96, 99]"
1,1,4,"[1, 2, 4, 5, 6, 10, 15, 18, 20, 25, 30, 37, 38, 40, 44, 46, 49, 51, 53, 54, 55, 56, 71, 81, 82, 84, 86, 89, 91, 92, 96, 97]"
2,1,5,"[1, 2, 4, 6, 10, 12, 13, 15, 17, 20, 21, 22, 25, 29, 31, 37, 38, 46, 49, 52, 54, 55, 58, 64, 67, 70, 81, 82, 84, 89, 91, 92, 94, 95]"
3,1,6,"[4, 10, 22, 34, 41, 50, 51, 52, 53, 55, 67, 77, 80, 82, 83, 84, 92, 97]"
4,1,7,"[1, 7, 10, 13, 17, 22, 30, 31, 41, 44, 49, 50, 52, 56, 77, 80, 81, 84, 91, 94, 96, 97, 99]"
5,1,8,"[7, 13, 15, 29, 34, 51, 64, 91]"
6,1,10,"[2, 4, 6, 13, 20, 36, 37, 38, 46, 54, 55, 66, 71, 77, 79, 81, 86, 89, 92, 96, 97, 99]"
7,1,11,"[4, 5, 7, 30, 34, 40, 44, 46, 49, 50, 64, 66, 89, 95, 96, 97]"
8,1,12,"[1, 2, 4, 5, 6, 7, 10, 15, 17, 18, 22, 25, 31, 34, 36, 37, 41, 42, 55, 58, 64, 66, 67, 71, 79, 80, 82, 83, 89, 91, 97, 99]"
9,1,13,"[2, 5, 13, 18, 21, 29, 30, 34, 36, 38, 40, 42, 46, 51, 56, 82, 86, 95, 99]"


In [8]:
import numpy as np

In [9]:
import pandas as pd

In [16]:
edf=pd.DataFrame({'source':[1,1,2,2],'target':[4,6,3,5]})
edf

,source,target
0,1,4
1,1,6
2,2,3
3,2,5


In [17]:
colegios=pd.DataFrame()
j=0
col={}
col[j]=[]

In [18]:
#i=3
#if True:
for i in edf.source.unique():
    #print(i)
    if True:
        if not i in col[j]: # new colegio!
            cl=edf[edf.source==i]
            # TODO loop in target
            if cl.target.values[0] not in col[j]: # neither source not target in old colegio:
                j=i
                col[j] =list( np.unique( np.concatenate ( ( cl.source.values,cl.target.values )  ) )  )
            else:
                col[j].append(i)

In [26]:
col.keys()

dict_keys([0, 1, 2])

In [31]:
cdf=pd.DataFrame()
cdf['colegios']=pd.Series(col)
cdf['source']=col.keys()
edf=edf.merge(cdf,on='source',how='left')

In [32]:
edf

,source,target,colegios
0,1,4,"[1, 4, 6]"
1,1,6,"[1, 4, 6]"
2,2,3,"[2, 3, 5]"
3,2,5,"[2, 3, 5]"


In [22]:
for i col.

{0: [], 1: [1, 4, 6], 2: [2, 3, 5]}

In [21]:
pd.DataFrame(col)

ValueError: arrays must all be same length

In [590]:
cl.source.values[0]

3

In [585]:
4 not in col[1]

False

In [580]:
cl=edf[edf.source==i]
cl

,source,target,intersect
168,3,4,"[39, 40, 44, 48, 56]"
169,3,25,"[29, 39, 40, 48, 56]"
170,3,31,"[29, 39, 40, 44, 56]"
171,3,33,"[29, 39, 44, 48, 56]"
172,3,42,"[29, 40, 44, 48, 56]"
173,3,51,"[29, 39, 40, 48, 56]"
174,3,59,"[29, 39, 40, 44, 48]"
175,3,61,"[29, 39, 44, 48, 56]"
176,3,68,"[39, 40, 44, 48, 56]"
177,3,99,"[29, 39, 40, 44, 48]"


In [568]:
edf[edf.source==1]

,source,target,intersect
0,1,2,"[2, 4, 18, 20, 25, 29, 31, 33, 34, 36, 40, 41, 51, 64, 81, 84, 86, 89, 91, 94, 95, 96, 99]"
1,1,4,"[1, 2, 4, 5, 6, 10, 15, 18, 20, 25, 30, 37, 38, 40, 44, 46, 49, 51, 53, 54, 55, 56, 71, 81, 82, 84, 86, 89, 91, 92, 96, 97]"
2,1,5,"[1, 2, 4, 6, 10, 12, 13, 15, 17, 20, 21, 22, 25, 29, 31, 37, 38, 46, 49, 52, 54, 55, 58, 64, 67, 70, 81, 82, 84, 89, 91, 92, 94, 95]"
3,1,6,"[4, 10, 22, 34, 41, 50, 51, 52, 53, 55, 67, 77, 80, 82, 83, 84, 92, 97]"
4,1,7,"[1, 7, 10, 13, 17, 22, 30, 31, 41, 44, 49, 50, 52, 56, 77, 80, 81, 84, 91, 94, 96, 97, 99]"
5,1,8,"[7, 13, 15, 29, 34, 51, 64, 91]"
6,1,10,"[2, 4, 6, 13, 20, 36, 37, 38, 46, 54, 55, 66, 71, 77, 79, 81, 86, 89, 92, 96, 97, 99]"
7,1,11,"[4, 5, 7, 30, 34, 40, 44, 46, 49, 50, 64, 66, 89, 95, 96, 97]"
8,1,12,"[1, 2, 4, 5, 6, 7, 10, 15, 17, 18, 22, 25, 31, 34, 36, 37, 41, 42, 55, 58, 64, 66, 67, 71, 79, 80, 82, 83, 89, 91, 97, 99]"
9,1,13,"[2, 5, 13, 18, 21, 29, 30, 34, 36, 38, 40, 42, 46, 51, 56, 82, 86, 95, 99]"


In [524]:
np.unique( np.concatenate ( ( cl.source.values,cl.target.values )  ) )

array([15, 16])

In [501]:
i not in col[0]

True

True

In [515]:
 not in cl[j]

KeyError: 0

In [480]:
edf=edf.append({'source':15,'target':16},ignore_index=True)

In [484]:
edf=edf[['source','target']].astype(int)

In [485]:
edf

,source,target
0,2,3
1,2,6
2,2,7
3,2,8
4,2,10
5,2,11
6,3,6
7,3,11
8,3,12
9,6,7


In [438]:
col=0
for j  in range( len([]) ):
    print(j)

In [427]:
i=3
if True:
    if i 
    cl=edf[edf.source==i]
    #col.append( np.unique( np.concatenate ( ( cl.source.values,cl.target.values )  ) ) )

In [431]:
3 in [[3,2]]

False

In [415]:
np.unique( np.concatenate( edf[edf.source==i].intersect.values ) )

array([ 1,  3,  4,  5,  7,  8,  9, 10, 11])

In [401]:
#df.to_csv('example_colegios.csv',index=False)

In [402]:
'''
# 'example_colegios.csv'
red               colegio 1             colegio 2
2
3,6,7,8,10,11 -> (2,3,6,7,8,10,11)      desconectados: 1,4,5,9,13
3 in colegio:
    True 
    6,11 True

    12 False -> (2,3,6,7,8,10,11,12)
6 in colegio:
    True
    7,8,11,12 True
7  in colegio:
    True:
    3,6,10,11,12 True

8 in colegio:
    True:
    11,12
    
... 
15 en colegio:
    False
    start colegio 3
'''

"\n# 'example_colegios.csv'\nred               colegio 1             colegio 2\n2\n3,6,7,8,10,11 -> (2,3,6,7,8,10,11)      desconectados: 4,5,9\n3 in colegio:\n    True \n    6,11 True\n\n    12 False -> (2,3,6,7,8,10,11,12)\n6 in colegio:\n    True\n    7,8,11,12 True\n7  in colegio:\n    True:\n    3,6,10,11,12 True\n\n8 in colegio:\n    True:\n    11,12\n    \n... \n15 en colegio:\n    False\n    start colegio 3\n"

In [403]:
ed

,source,target,intersect
0,1,2,"[5, 8]"
1,1,3,"[5, 8]"
2,1,6,[5]
3,1,8,"[5, 8]"
4,1,10,[8]
5,1,11,[5]
6,1,12,[5]
7,2,3,"[3, 5, 7, 8, 9, 10]"
8,2,5,"[1, 3, 11]"
9,2,6,"[4, 5, 7, 9, 10, 11]"


In [377]:
if True:
    i=2
    #remove i=2
    df_not_i=df.drop(df.index[[i]])
    df_not_i['intersect']=df_not_i.references.map(lambda x: check_intersect1d(x,df.loc[i,'references']))
    df_not_i=df_not_i[df_not_i.intersect.map(len)>0]
    df_not_i=df_not_i.rename_axis({'recid':'target'}, axis = 'columns')
    df_not_i['source']=df.loc[i,'recid']
    # switch source, target order and remove if already in edges
    df_not_i=df_not_i[ ( df_not_i.target.map( lambda x: [x] )+df_not_i.source.map( lambda x: [x] )
          ).map(lambda x: check_not_ref(x,edges) )]    
    
    # list of edges: [ [source, target],... ]
    edges=edges+np.stack(( df_not_i.source.values,df_not_i.target.values),axis=1 ).tolist()    
    #Store Edges DataFrame
    ed=ed.append( df_not_i[['source','target','intersect']] ).reset_index(drop=True)

ed    

,source,target,intersect
0,1,2,"[5, 8]"
1,1,3,"[5, 8]"
2,1,6,[5]
3,1,8,"[5, 8]"
4,1,10,[8]
5,1,11,[5]
6,1,12,[5]
7,2,3,"[3, 5, 7, 8, 9, 10]"
8,2,5,"[1, 3, 11]"
9,2,6,"[4, 5, 7, 9, 10, 11]"


In [366]:
edges

[[1, 2],
 [1, 3],
 [1, 6],
 [1, 8],
 [1, 10],
 [1, 11],
 [1, 12],
 [2, 3],
 [2, 5],
 [2, 6],
 [2, 7],
 [2, 8],
 [2, 9],
 [2, 10],
 [2, 11],
 [2, 12],
 [2, 13]]

,target,references,intersect,source,ST
2,3,"[3, 5, 7, 8, 9, 10, 12]","[3, 5, 7, 8, 9, 10]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
4,5,"[1, 3, 6, 11]","[1, 3, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
5,6,"[2, 4, 5, 7, 9, 10, 11, 12]","[4, 5, 7, 9, 10, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
6,7,"[1, 2, 3, 4, 10, 11, 12]","[1, 3, 4, 10, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
7,8,"[2, 4, 5, 6, 7, 8, 11]","[4, 5, 7, 8, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
8,9,[1],[1],2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
9,10,"[1, 3, 4, 8, 10]","[1, 3, 4, 8, 10]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
10,11,"[3, 4, 5, 6, 7, 10, 11]","[3, 4, 5, 7, 10, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
11,12,"[2, 3, 5, 6, 10, 12]","[3, 5, 10]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
12,13,[11],[11],2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"


In [318]:
(df_not_i.target.astype(str)+'::'+df_not_i.source.astype(str) ).str.split('::')

SyntaxError: unexpected EOF while parsing (<ipython-input-318-823e03b869c7>, line 1)

In [312]:
( df_not_i.target.astype(str)+'::'+df_not_i.source.astype(str) ).str.split('::').values

array([list(['1', '2']), list(['3', '2']), list(['5', '2']),
       list(['6', '2']), list(['7', '2']), list(['8', '2']),
       list(['9', '2']), list(['10', '2']), list(['11', '2']),
       list(['12', '2']), list(['13', '2'])], dtype=object)

In [308]:
def check_ref(x,ref):
    return x in ref
( df_not_i.target.astype(str)+'::'+df_not_i.source.astype(str) ).str.split('::').map(lambda x: check_ref(x,[[1,2]] ))
           #np.stack(( ed.source.values,ed.target.values),axis=1 )          ))

0     False
2     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
dtype: bool

In [310]:
check_ref([1,2] , [[1,2]])

True

In [287]:
df_not_i.assign(  TS= pd.Series( {'TS':np.stack(( df_not_i.target.values,df_not_i.source.values),axis=1 )}).values )

ValueError: Length of values does not match length of index

In [242]:
1+1

2

In [240]:
np.stack?